In [1]:
import json,csv
from kafka import KafkaProducer
import time

In [2]:
def producer(file,sleep_time=0.00015):
    producer = KafkaProducer( bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
                               value_serializer=lambda v: json.dumps(v).encode('utf-8'))
    path ='./Data/'
    with open(path+file+'.csv') as file:
        reader = csv.DictReader(file, delimiter=",")
        i = 0
        start = time.time()
        for row in reader:
            
            row["HEAD"] = int(row["HEAD"])
            row["FPGA"] = int(row["FPGA"])
            row["TDC_CHANNEL"] = int(row["TDC_CHANNEL"])
            row["ORBIT_CNT"] = float(row["ORBIT_CNT"])
            row["BX_COUNTER"] = int(row["BX_COUNTER"])
            row["TDC_MEAS"] = float(row["TDC_MEAS"])
            producer.send(topic='Experiment_measurements', value=row)
            #producer.send(topic='Experiment_measurements', value=json.dumps(row))
            #producer.flush()
            time.sleep(sleep_time) #This was adjusted "by eye" to send 1000 elements/second
            i += 1
            if (i==1000):
                i = 0
                end = time.time()
                print(f"The time to print elements is:",end-start)
                start = end
                

In [4]:
KAFKA_BOOTSTRAP_SERVERS = 'slave01:9092'

for file in ['data_000000']:
    print(file)
    producer(file)

data_000000
The time to print elements is: 0.7695696353912354
The time to print elements is: 0.9652070999145508
The time to print elements is: 0.8954508304595947
The time to print elements is: 0.7748491764068604
The time to print elements is: 0.6550118923187256
The time to print elements is: 0.6565852165222168
The time to print elements is: 0.6590912342071533
The time to print elements is: 0.6555845737457275
The time to print elements is: 0.711968183517456
The time to print elements is: 0.7276163101196289
The time to print elements is: 0.9700219631195068
The time to print elements is: 0.9223513603210449
The time to print elements is: 0.8512763977050781
The time to print elements is: 0.6621718406677246
The time to print elements is: 0.6475570201873779
The time to print elements is: 0.6539936065673828
The time to print elements is: 0.7028560638427734
The time to print elements is: 0.7870209217071533
The time to print elements is: 0.828453540802002
The time to print elements is: 0.7848134

The time to print elements is: 0.8010284900665283
The time to print elements is: 0.7536647319793701
The time to print elements is: 0.7768998146057129
The time to print elements is: 0.7876560688018799
The time to print elements is: 0.6562674045562744
The time to print elements is: 0.6565718650817871
The time to print elements is: 0.6653585433959961
The time to print elements is: 0.6723148822784424
The time to print elements is: 0.7745838165283203
The time to print elements is: 0.7896895408630371
The time to print elements is: 0.7291696071624756
The time to print elements is: 0.757012128829956
The time to print elements is: 0.7471237182617188
The time to print elements is: 0.735842227935791
The time to print elements is: 0.7013964653015137
The time to print elements is: 0.6795213222503662
The time to print elements is: 0.7577805519104004
The time to print elements is: 0.8572454452514648
The time to print elements is: 0.7952635288238525
The time to print elements is: 0.8282980918884277
Th

The time to print elements is: 0.893585205078125
The time to print elements is: 1.007209062576294
The time to print elements is: 0.9619565010070801
The time to print elements is: 0.7874925136566162
The time to print elements is: 0.781461238861084
The time to print elements is: 0.8004124164581299
The time to print elements is: 1.0829527378082275
The time to print elements is: 0.8941597938537598
The time to print elements is: 0.9125421047210693
The time to print elements is: 0.8939499855041504
The time to print elements is: 0.8609721660614014
The time to print elements is: 0.7515387535095215
The time to print elements is: 0.8504951000213623
The time to print elements is: 1.0227246284484863
The time to print elements is: 0.9071927070617676
The time to print elements is: 1.004918098449707
The time to print elements is: 0.9258923530578613
The time to print elements is: 0.8128397464752197
The time to print elements is: 0.7885589599609375
The time to print elements is: 0.9408087730407715
The 

The time to print elements is: 0.6780288219451904
The time to print elements is: 0.7270810604095459
The time to print elements is: 0.6982746124267578
The time to print elements is: 0.6848950386047363
The time to print elements is: 0.6300275325775146
The time to print elements is: 0.629246711730957
The time to print elements is: 0.6689834594726562
The time to print elements is: 0.8325934410095215
The time to print elements is: 0.7364087104797363
The time to print elements is: 0.7339038848876953
The time to print elements is: 0.7606818675994873
The time to print elements is: 0.62788987159729
The time to print elements is: 0.6376779079437256
The time to print elements is: 0.6389095783233643
The time to print elements is: 0.6431372165679932
The time to print elements is: 0.7544918060302734
The time to print elements is: 0.8008334636688232
The time to print elements is: 0.6820540428161621
The time to print elements is: 0.7741367816925049
The time to print elements is: 0.6899256706237793
The

The time to print elements is: 0.6146135330200195
The time to print elements is: 0.6812927722930908
The time to print elements is: 0.8327748775482178
The time to print elements is: 0.7066020965576172
The time to print elements is: 0.8113038539886475
The time to print elements is: 0.8449718952178955
The time to print elements is: 1.0265848636627197
The time to print elements is: 1.014192819595337
The time to print elements is: 0.9564609527587891
The time to print elements is: 0.8463249206542969
The time to print elements is: 0.7898776531219482
The time to print elements is: 0.7571797370910645
The time to print elements is: 0.6832282543182373
The time to print elements is: 0.6269936561584473
The time to print elements is: 0.6246869564056396
The time to print elements is: 0.6220853328704834
The time to print elements is: 0.6506752967834473
The time to print elements is: 0.7064230442047119
The time to print elements is: 0.7611434459686279
The time to print elements is: 0.6927919387817383
T

The time to print elements is: 0.661698579788208
The time to print elements is: 0.6294410228729248
The time to print elements is: 0.6254055500030518
The time to print elements is: 0.6252005100250244
The time to print elements is: 0.6764700412750244
The time to print elements is: 0.7571008205413818
The time to print elements is: 0.8367495536804199
The time to print elements is: 0.7625527381896973
The time to print elements is: 0.7840123176574707
The time to print elements is: 0.6552352905273438
The time to print elements is: 0.6427514553070068
The time to print elements is: 0.6454594135284424
The time to print elements is: 0.6695811748504639
The time to print elements is: 0.7176439762115479
The time to print elements is: 0.7931435108184814
The time to print elements is: 0.7085757255554199
The time to print elements is: 0.7735915184020996
The time to print elements is: 0.6728272438049316
The time to print elements is: 0.6271708011627197
The time to print elements is: 0.6152994632720947
T

The time to print elements is: 0.7016632556915283
The time to print elements is: 0.6117167472839355
The time to print elements is: 0.60219407081604
The time to print elements is: 0.6241686344146729
The time to print elements is: 0.6766312122344971
The time to print elements is: 0.8120274543762207
The time to print elements is: 0.7882626056671143
The time to print elements is: 0.7839362621307373
The time to print elements is: 0.7467060089111328
The time to print elements is: 0.6286351680755615
The time to print elements is: 0.6127541065216064
The time to print elements is: 0.6105258464813232
The time to print elements is: 0.6340534687042236
The time to print elements is: 0.6291017532348633
The time to print elements is: 0.8786594867706299
The time to print elements is: 0.675987958908081
The time to print elements is: 0.7217154502868652
The time to print elements is: 0.7191460132598877
The time to print elements is: 0.6062207221984863
The time to print elements is: 0.6089181900024414
The

The time to print elements is: 0.7428045272827148
The time to print elements is: 0.6931297779083252
The time to print elements is: 0.625629186630249
The time to print elements is: 0.6154913902282715
The time to print elements is: 0.6155951023101807
The time to print elements is: 0.6201438903808594
The time to print elements is: 0.6787636280059814
The time to print elements is: 0.743182897567749
The time to print elements is: 0.7366883754730225
The time to print elements is: 0.7406227588653564
The time to print elements is: 0.7157223224639893
The time to print elements is: 0.6176552772521973
The time to print elements is: 0.6139817237854004
The time to print elements is: 0.6150648593902588
The time to print elements is: 0.6338100433349609
The time to print elements is: 0.7500379085540771
The time to print elements is: 0.8516418933868408
The time to print elements is: 0.7569952011108398
The time to print elements is: 0.7373802661895752
The time to print elements is: 0.6726462841033936
Th

In [42]:
producer.close()

AttributeError: 'function' object has no attribute 'close'